# 1. Import and Install Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
!pip install opencv-python mediapipe sklearn 

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
   # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
   #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
   #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
  #                           ) 
    # Draw pose connections
  #  mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
  #                           mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
   #                          mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    #                         ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [7]:
len(results.left_hand_landmarks.landmark)

21

In [8]:
results

mediapipe.python.solution_base.SolutionOutputs

In [9]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [11]:
len(results.left_hand_landmarks.landmark)

21

In [12]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [13]:
#pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
#face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [14]:
def extract_keypoints(results):
   # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
   # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [15]:
result_test = extract_keypoints(results)

In [16]:
result_test

array([ 7.87672579e-01,  6.81258321e-01,  3.26084574e-07,  7.40069807e-01,
        6.44009471e-01, -1.68219414e-02,  7.00810611e-01,  5.92855334e-01,
       -2.72305366e-02,  6.72093511e-01,  5.53592741e-01, -3.69381793e-02,
        6.46966457e-01,  5.24400175e-01, -4.67730239e-02,  7.38561094e-01,
        5.08556843e-01, -1.39263747e-02,  7.25433826e-01,  4.41984415e-01,
       -2.88436338e-02,  7.18062282e-01,  4.01438773e-01, -4.17890809e-02,
        7.13603020e-01,  3.64195585e-01, -5.20103946e-02,  7.71480441e-01,
        5.02661109e-01, -1.90204177e-02,  7.69882381e-01,  4.28248554e-01,
       -3.16552445e-02,  7.69783318e-01,  3.81676972e-01, -4.34835106e-02,
        7.69231081e-01,  3.39705586e-01, -5.30230142e-02,  8.02312851e-01,
        5.12676477e-01, -2.71682162e-02,  8.07827294e-01,  4.42046672e-01,
       -4.26989645e-02,  8.09352756e-01,  3.97255957e-01, -5.32103367e-02,
        8.08862209e-01,  3.55338454e-01, -6.10548742e-02,  8.30969930e-01,
        5.35485804e-01, -

In [17]:
468*3+33*4+21*3+21*3

1662

In [18]:
np.save('0', result_test)

In [19]:
np.load('0.npy')

array([ 7.87672579e-01,  6.81258321e-01,  3.26084574e-07,  7.40069807e-01,
        6.44009471e-01, -1.68219414e-02,  7.00810611e-01,  5.92855334e-01,
       -2.72305366e-02,  6.72093511e-01,  5.53592741e-01, -3.69381793e-02,
        6.46966457e-01,  5.24400175e-01, -4.67730239e-02,  7.38561094e-01,
        5.08556843e-01, -1.39263747e-02,  7.25433826e-01,  4.41984415e-01,
       -2.88436338e-02,  7.18062282e-01,  4.01438773e-01, -4.17890809e-02,
        7.13603020e-01,  3.64195585e-01, -5.20103946e-02,  7.71480441e-01,
        5.02661109e-01, -1.90204177e-02,  7.69882381e-01,  4.28248554e-01,
       -3.16552445e-02,  7.69783318e-01,  3.81676972e-01, -4.34835106e-02,
        7.69231081e-01,  3.39705586e-01, -5.30230142e-02,  8.02312851e-01,
        5.12676477e-01, -2.71682162e-02,  8.07827294e-01,  4.42046672e-01,
       -4.26989645e-02,  8.09352756e-01,  3.97255957e-01, -5.32103367e-02,
        8.08862209e-01,  3.55338454e-01, -6.10548742e-02,  8.30969930e-01,
        5.35485804e-01, -

# 4. Setup Folders for Collection

In [20]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'nice'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [21]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [32]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [22]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [23]:
label_map = {label:num for num, label in enumerate(actions)}

In [24]:
label_map

{'hello': 0, 'thanks': 1, 'nice': 2}

In [25]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [26]:
np.array(sequences).shape

(90, 30, 126)

In [27]:
np.array(labels).shape

(90,)

In [28]:
X = np.array(sequences)

In [29]:
X.shape

(90, 30, 126)

In [30]:
y = to_categorical(labels).astype(int)

In [31]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [33]:
y_test.shape

(5, 3)

# 7. Build and Train LSTM Neural Network

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [35]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [36]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [37]:
res = [.7, 0.2, 0.1]

In [38]:
actions[np.argmax(res)]

'hello'

In [39]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [43]:
model.fit(X_train, y_train, epochs=1500, callbacks=[tb_callback])

Epoch 1/1500
3/3 [==============================] - 3s 55ms/step - loss: 1.0806 - categorical_accuracy: 0.5059
Epoch 2/1500
3/3 [==============================] - 0s 49ms/step - loss: 0.9855 - categorical_accuracy: 0.5294
Epoch 3/1500
3/3 [==============================] - 0s 50ms/step - loss: 0.9562 - categorical_accuracy: 0.5647
Epoch 4/1500
3/3 [==============================] - 0s 54ms/step - loss: 0.7624 - categorical_accuracy: 0.7882
Epoch 5/1500
3/3 [==============================] - 0s 66ms/step - loss: 1.5300 - categorical_accuracy: 0.6000
Epoch 6/1500
3/3 [==============================] - 0s 46ms/step - loss: 0.9201 - categorical_accuracy: 0.7059
Epoch 7/1500
3/3 [==============================] - 0s 56ms/step - loss: 0.8593 - categorical_accuracy: 0.7059
Epoch 8/1500
3/3 [==============================] - 0s 51ms/step - loss: 0.8102 - categorical_accuracy: 0.7059
Epoch 9/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.7427 - categorical_accuracy: 0.7412
E

3/3 [==============================] - 0s 66ms/step - loss: 0.1176 - categorical_accuracy: 0.9765
Epoch 75/1500
3/3 [==============================] - 0s 64ms/step - loss: 0.0972 - categorical_accuracy: 0.9765
Epoch 76/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.0890 - categorical_accuracy: 0.9765
Epoch 77/1500
3/3 [==============================] - 0s 64ms/step - loss: 0.0647 - categorical_accuracy: 0.9882
Epoch 78/1500
3/3 [==============================] - 0s 74ms/step - loss: 0.0881 - categorical_accuracy: 0.9765
Epoch 79/1500
3/3 [==============================] - 0s 63ms/step - loss: 0.0580 - categorical_accuracy: 0.9765
Epoch 80/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.0651 - categorical_accuracy: 0.9765
Epoch 81/1500
3/3 [==============================] - 0s 63ms/step - loss: 0.1679 - categorical_accuracy: 0.9647
Epoch 82/1500
3/3 [==============================] - 0s 63ms/step - loss: 0.1438 - categorical_accuracy: 0.9647
Epoch 

3/3 [==============================] - 0s 68ms/step - loss: 0.0520 - categorical_accuracy: 0.9882
Epoch 147/1500
3/3 [==============================] - 0s 62ms/step - loss: 0.0518 - categorical_accuracy: 0.9882
Epoch 148/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.0521 - categorical_accuracy: 0.9882
Epoch 149/1500
3/3 [==============================] - 0s 62ms/step - loss: 0.0518 - categorical_accuracy: 0.9882
Epoch 150/1500
3/3 [==============================] - 0s 50ms/step - loss: 0.0511 - categorical_accuracy: 0.9882
Epoch 151/1500
3/3 [==============================] - 0s 53ms/step - loss: 0.0509 - categorical_accuracy: 0.9882
Epoch 152/1500
3/3 [==============================] - 0s 64ms/step - loss: 0.0512 - categorical_accuracy: 0.9882
Epoch 153/1500
3/3 [==============================] - 0s 57ms/step - loss: 0.0508 - categorical_accuracy: 0.9882
Epoch 154/1500
3/3 [==============================] - 0s 69ms/step - loss: 0.0504 - categorical_accuracy: 0.988

3/3 [==============================] - 0s 67ms/step - loss: 0.0499 - categorical_accuracy: 0.9882
Epoch 219/1500
3/3 [==============================] - 0s 63ms/step - loss: 0.0446 - categorical_accuracy: 0.9882
Epoch 220/1500
3/3 [==============================] - 0s 65ms/step - loss: 1.5483 - categorical_accuracy: 0.9294
Epoch 221/1500
3/3 [==============================] - 0s 63ms/step - loss: 0.2863 - categorical_accuracy: 0.9176
Epoch 222/1500
3/3 [==============================] - 0s 56ms/step - loss: 0.2447 - categorical_accuracy: 0.8706
Epoch 223/1500
3/3 [==============================] - 0s 57ms/step - loss: 0.3169 - categorical_accuracy: 0.8471
Epoch 224/1500
3/3 [==============================] - 0s 52ms/step - loss: 0.4054 - categorical_accuracy: 0.8471
Epoch 225/1500
3/3 [==============================] - 0s 54ms/step - loss: 0.2894 - categorical_accuracy: 0.8471
Epoch 226/1500
3/3 [==============================] - 0s 69ms/step - loss: 0.2793 - categorical_accuracy: 0.952

3/3 [==============================] - 0s 65ms/step - loss: 0.2865 - categorical_accuracy: 0.9647
Epoch 291/1500
3/3 [==============================] - 0s 54ms/step - loss: 0.2248 - categorical_accuracy: 0.9412
Epoch 292/1500
3/3 [==============================] - 0s 74ms/step - loss: 0.2157 - categorical_accuracy: 0.9765
Epoch 293/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.1751 - categorical_accuracy: 0.9647
Epoch 294/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.1742 - categorical_accuracy: 0.9647
Epoch 295/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.1668 - categorical_accuracy: 0.9765
Epoch 296/1500
3/3 [==============================] - 0s 68ms/step - loss: 0.1629 - categorical_accuracy: 0.9765
Epoch 297/1500
3/3 [==============================] - 0s 62ms/step - loss: 0.1651 - categorical_accuracy: 0.9647
Epoch 298/1500
3/3 [==============================] - 0s 64ms/step - loss: 0.1621 - categorical_accuracy: 0.964

3/3 [==============================] - 0s 64ms/step - loss: 0.0460 - categorical_accuracy: 0.9882
Epoch 363/1500
3/3 [==============================] - 0s 72ms/step - loss: 0.0389 - categorical_accuracy: 0.9882
Epoch 364/1500
3/3 [==============================] - 0s 48ms/step - loss: 0.0479 - categorical_accuracy: 0.9882
Epoch 365/1500
3/3 [==============================] - 0s 58ms/step - loss: 0.0395 - categorical_accuracy: 1.0000
Epoch 366/1500
3/3 [==============================] - 0s 70ms/step - loss: 0.0293 - categorical_accuracy: 1.0000
Epoch 367/1500
3/3 [==============================] - 0s 67ms/step - loss: 0.0373 - categorical_accuracy: 0.9882
Epoch 368/1500
3/3 [==============================] - 0s 65ms/step - loss: 0.0281 - categorical_accuracy: 1.0000
Epoch 369/1500
3/3 [==============================] - 0s 70ms/step - loss: 0.0463 - categorical_accuracy: 0.9882
Epoch 370/1500
3/3 [==============================] - 0s 64ms/step - loss: 0.0762 - categorical_accuracy: 0.988

3/3 [==============================] - 0s 108ms/step - loss: 0.0144 - categorical_accuracy: 1.0000
Epoch 435/1500
3/3 [==============================] - 0s 71ms/step - loss: 0.0238 - categorical_accuracy: 0.9882
Epoch 436/1500
3/3 [==============================] - 0s 65ms/step - loss: 0.0481 - categorical_accuracy: 0.9765
Epoch 437/1500
3/3 [==============================] - 0s 64ms/step - loss: 0.0637 - categorical_accuracy: 0.9882
Epoch 438/1500
3/3 [==============================] - 0s 65ms/step - loss: 0.0337 - categorical_accuracy: 0.9882
Epoch 439/1500
3/3 [==============================] - 0s 64ms/step - loss: 0.1480 - categorical_accuracy: 0.9882
Epoch 440/1500
3/3 [==============================] - 0s 60ms/step - loss: 0.0288 - categorical_accuracy: 1.0000
Epoch 441/1500
3/3 [==============================] - 0s 66ms/step - loss: 0.0357 - categorical_accuracy: 0.9882
Epoch 442/1500
3/3 [==============================] - 0s 62ms/step - loss: 0.0340 - categorical_accuracy: 0.98

3/3 [==============================] - 0s 62ms/step - loss: 0.0013 - categorical_accuracy: 1.0000
Epoch 507/1500
3/3 [==============================] - 0s 61ms/step - loss: 0.7457 - categorical_accuracy: 0.9176
Epoch 508/1500
3/3 [==============================] - 0s 51ms/step - loss: 61.6727 - categorical_accuracy: 0.5647
Epoch 509/1500
3/3 [==============================] - 0s 52ms/step - loss: 296.4630 - categorical_accuracy: 0.5294
Epoch 510/1500
3/3 [==============================] - 0s 90ms/step - loss: 50.1617 - categorical_accuracy: 0.6941
Epoch 511/1500
3/3 [==============================] - 0s 67ms/step - loss: 2.5056 - categorical_accuracy: 0.6118
Epoch 512/1500
3/3 [==============================] - 0s 51ms/step - loss: 1.4620 - categorical_accuracy: 0.6588
Epoch 513/1500
3/3 [==============================] - 0s 51ms/step - loss: 1.5037 - categorical_accuracy: 0.6588
Epoch 514/1500
3/3 [==============================] - 0s 58ms/step - loss: 2.3140 - categorical_accuracy: 0

3/3 [==============================] - 0s 58ms/step - loss: 0.2050 - categorical_accuracy: 0.9882
Epoch 579/1500
3/3 [==============================] - 0s 94ms/step - loss: 0.2175 - categorical_accuracy: 0.9412
Epoch 580/1500
3/3 [==============================] - 0s 72ms/step - loss: 0.2072 - categorical_accuracy: 0.9882
Epoch 581/1500
3/3 [==============================] - 0s 77ms/step - loss: 0.1516 - categorical_accuracy: 0.9882
Epoch 582/1500
3/3 [==============================] - 0s 70ms/step - loss: 0.1549 - categorical_accuracy: 0.9882
Epoch 583/1500
3/3 [==============================] - 0s 48ms/step - loss: 0.1450 - categorical_accuracy: 0.9882
Epoch 584/1500
3/3 [==============================] - 0s 86ms/step - loss: 0.1612 - categorical_accuracy: 0.9882
Epoch 585/1500
3/3 [==============================] - 0s 73ms/step - loss: 0.1193 - categorical_accuracy: 0.9882
Epoch 586/1500
3/3 [==============================] - 0s 75ms/step - loss: 0.0984 - categorical_accuracy: 0.988

KeyboardInterrupt: 

In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 203,459
Trainable params: 203,459
Non-trai

# 8. Make Predictions

In [45]:
res = model.predict(X_test)

In [46]:
actions[np.argmax(res[2])]

'hello'

In [47]:
actions[np.argmax(y_test[2])]

'hello'

# 9. Save Weights

In [48]:
model.save('action.h5')

In [49]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [50]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [51]:
yhat = model.predict(X_test)

In [52]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [53]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 0],
        [0, 3]],

       [[4, 0],
        [0, 1]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [54]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [55]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [56]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hell

<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello

<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
nice
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<

In [81]:
res[np.argmax(res)] > threshold

True